# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Henning  ->  T. Henning  |  ['T. Henning']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Liu  ->  J. Liu  |  ['J. Liu']


J. Li  ->  J. Li  |  ['J. Li']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 77 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.11761


extracting tarball to tmp_2508.11761...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_d

...

extracting tarball to tmp_2508.13136...

 done.
Retrieving document from  https://arxiv.org/e-print/2508.13145


extracting tarball to tmp_2508.13145...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2508.13145/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/occurrence_table' from 'tmp_2508.13145/tables/occurrence_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/TOI-5293A_RVtable' from 'tmp_2508.13145/tables/TOI-5293A_RVtable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.11761-b31b1b.svg)](https://arxiv.org/abs/2508.11761) | **MINDS. Strong oxygen depletion in the inner regions of a very low-mass star disk?**  |
|| J. Kanwar, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *accepted for publication in AandA*|
|**Abstract**|            JWST is discovering a plethora of species in planet-forming disks around very low-mass stars such as C2H2, C6H6, C4H2, CH3 etc. The column densities of these species retrieved from 0D slab models are very large, e.g. of the order of $10^{20}$\,cm$^{-2}$. This is indicating a carbon-dominated chemistry in a gas with a high C/O ratio. The disk around 2MASS-J1605321-1993159 (M4.5) is one such source showing a molecular pseudo-continuum of C2H2. Still two oxygen-bearing molecules, CO and CO2 are also detected in this source. We aim to take the next step beyond 0D slab models to interpret the spectrum. We examine whether 2D thermo-chemical disk models can produce the large inferred column densities of \ce{C2H2} in the inner regions of the disk and produce a pseudo-continuum in the mid-IR spectrum. We also want to constrain whether depletion of oxygen or enrichment of carbon is causing the high C/O ratio triggering a carbon-dominated chemistry. We utilize the radiative thermo-chemical disk model P{\tiny RO}D{\tiny I}M{\tiny O} to identify a disk structure which is capable of producing the observed molecular emission of species such as CO, CO2, C2H2, and H2O simultaneously. The spectrum is generated using the fast line tracer FLiTs. We derive the gas temperature $\langle T \rangle$, column density $\langle$ log$_{\rm {10}} N\rangle$ and the emitting area $\langle r_{\rm{1}} - r_{\rm{2}} \rangle$ for these molecules from the 2D disk model and compare them to the parameters retrieved originally from 0D slab models. We use the different effect that changing the O or C abundance has on CO and C2H2 respectively to discriminate between O depletion and C enhancement. We find that a disk structure characterised by the presence of a gap can best explain the observations. The inner disk is strongly depleted in dust, especially small grains ($<5\,\mu$m), and ..         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.12601-b31b1b.svg)](https://arxiv.org/abs/2508.12601) | **A Tale of Two Sightlines: Comparison of Hydrocarbon Dust Absorption Bands toward Cygnus OB2-12 and the Galactic Center**  |
|| Y. J. Pendleton, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *28 pages, 7 figures, accepted by ApJ*|
|**Abstract**|            Infrared spectra of hydrocarbon dust absorption bands toward the bright hypergiant Cygnus OB2-12 are compared to published spectra of the Quintuplet Cluster, a sightline to the Galactic center. The Cyg OB2-12 data include a new ground-based 2.86-3.70 microns spectrum and a previously published, but here further analyzed, spectrum of the 5.50-7.34 microns region. Higher spectral resolution data for the Cyg OB2-12 sightline in the 3 micron region allows a detailed comparison of the 3.4 micron aliphatic bands to those observed toward the Quintuplet. Despite differences in interstellar environments along each sightline, strong similarities are observed in the central wavelengths and relative strengths for bands at 3.3, 3.4, 5.85, 6.2, and 6.85 microns. Analysis of these bands, produced by aromatic, aliphatic, olefinic, hydrogenated, and oxygenated components, shows that carbonaceous dust is a significant component of the diffuse interstellar medium, second in abundance only to silicates, and is primarily aromatic in nature. The grains producing these bands likely consist of large aromatic carbon cores with thin aliphatic mantles composed of hydrogenated amorphous carbon (HAC). Laboratory analog spectra reproduce the observed aliphatic absorption bands well, supporting the presence of such mantles. We present evidence that the carriers of both the 3.4 micron aliphatic and the 3.3 micron aromatic bands reside exclusively in the diffuse ISM, and that the 3.3 micron bands observed in the diffuse ISM differ from those seen in dense clouds, implying chemically distinct carriers.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.13145-b31b1b.svg)](https://arxiv.org/abs/2508.13145) | **Aligned Stellar Obliquities for Two Hot Jupiter-hosting M Dwarfs Revealed by MAROON-X: Implications for Hot Jupiter Formation**  |
|| D. Weisserman, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *Resubmitted to The Astronomical Journal following a positive referee report. 21 pages, 5 figures, 5 tables. Our spectroscopic time series (i.e. Tables 3 & 4) are included as csv files in the arXiv source files*|
|**Abstract**|            Hot Jupiters (HJs) are $2-3\times$ less common around early M dwarfs than around AFGK stars, suggesting that HJs may form and/or migrate via distinct pathways around different types of stars. One source of insight into HJ formation mechanisms is to trace their dynamical histories through measurements of host stellar obliquities via the Rossiter-McLaughlin (RM) effect. Here we present measurements of the RM effect for the HJs TOI-3714 b and TOI-5293 A b using the Gemini-North/MAROON-X spectrograph. Our measurements represent just the second and third hot Jupiters around M dwarfs (HJMD) with a detection of the RM effect. We find that both systems are well-aligned with sky-projected obliquities of $\lambda = 21^{+14}_{-11}$$\mathrm{^{\circ}}$ and $-12^{+19}_{-14}$$\mathrm{^{\circ}}$ and deprojected obliquities of $\psi = 26^{+11}_{-10}$$\mathrm{^{\circ}}$ and $24^{+11}_{-10}$$\mathrm{^{\circ}}$ for TOI-3714 and TOI-5293 A, respectively. Both stars are in wide binary systems. We refine the stellar parameters by decontaminating their unresolved $K_s$-band photometry and constrain the binary orbits using Gaia DR3 astrometry. We find that the minimum mutual inclination of the planet and binary companion in the TOI-5293 system is sufficiently large to drive Kozai-Lidov (KL) migration while the result for TOI-3714 is inconclusive. We present a population-level analysis of HJs around AFGK versus early M dwarfs and argue that KL migration is more efficient around the latter, which is expected to produce misaligned stellar obliquities in HJMD systems in the absence of efficient tidal damping. The emerging population of well-aligned HJMD hosts supports the expectation that M dwarfs, with their deep convective envelopes, do efficiently dampen misaligned obliquities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.13015-b31b1b.svg)](https://arxiv.org/abs/2508.13015) | **Size Distributions of Arcsecond-Scale Properties of Solar Flare Ribbons**  |
|| Y. Zhang, R. Liu, W. Wang, <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *Accepted for publication in The Astrophysical Journal 12 pages, 5 figures, 3 tables*|
|**Abstract**|            Solar flare ribbons are believed to map the footpoints of newly reconnected magnetic flux tubes, therefore shedding light on the reconnecting current sheet, which is rarely observed by direct imaging or spectroscopy. Here we study the detailed evolution of flare ribbons down to the arcsecond scale for 10 flares characterized by the classic double ribbons. Identifying the flaring pixels by combining the intensity variances of the UV filter ratio and intensity threshold, we found that the waiting time distributions of the flaring pixels are well described by power laws, distinct from those in the preflare or quiet-Sun regions, and that the power-law slopes are generally consistent with those predicted by nonstationary Poisson processes in the nonlinear regime or by the 2D/3D self-organized criticality (SOC) model. The size distributions for flaring duration also follow power laws but the slopes are more scattered. In contrast, the size distributions for other parameters, including peak intensity, energy, and radial magnetic field strength of the flaring pixels, deviate from power laws, and the estimated slopes significantly differ from the SOC predictions. These results suggest that a nonstationary Poisson process or an avalanche-like process might be ongoing in the temporal dimension in the reconnecting current sheet, but in other aspects, e.g., space- and energy-wise, the avalanche is likely modulated by other physical processes or the fine structures of the reconnecting current sheet.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.13136-b31b1b.svg)](https://arxiv.org/abs/2508.13136) | **The ALPINE-CRISTAL-JWST survey: spatially resolved star formation relations at $z\sim5$**  |
|| C. Accard, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *20 pages, 13 figures*|
|**Abstract**|            Star formation governs galaxy evolution, shaping stellar mass assembly and gas consumption across cosmic time. The Kennicutt-Schmidt (KS) relation, linking star formation rate (SFR) and gas surface densities, is fundamental to understand star formation regulation, yet remains poorly constrained at $z > 2$ due to observational limitations and uncertainties in locally calibrated gas tracers. The [CII] $158 {\rm \mu m}$ line has recently emerged as a key probe of the cold ISM and star formation in the early Universe. We investigate whether the resolved [CII]-SFR and KS relations established at low redshift remain valid at $4 < z < 6$ by analysing 13 main-sequence galaxies from the ALPINE and CRISTAL surveys, using multi-wavelength data (HST, JWST, ALMA) at $\sim2$ kpc resolution. We perform pixel-by-pixel spectral energy distribution (SED) modelling with CIGALE on resolution-homogenised images. We develop a statistical framework to fit the [CII]-SFR relation that accounts for pixel covariance and compare our results to classical fitting methods. We test two [CII]-to-gas conversion prescriptions to assess their impact on inferred gas surface densities and depletion times. We find a resolved [CII]-SFR relation with a slope of $0.87 \pm 0.15$ and intrinsic scatter of $0.19 \pm 0.03$ dex, which is shallower and tighter than previous studies at $z\sim5$. The resolved KS relation is highly sensitive to the [CII]-to-gas conversion factor: using a fixed global $\alpha_{\rm [CII]}$ yields depletion times of $0.5$-$1$ Gyr, while a surface brightness-dependent $W_{\rm [CII]}$, places some galaxies with high gas density in the starburst regime ($<0.1$ Gyr). Future inputs from both simulations and observations are required to better understand how the [CII]-to-gas conversion factor depends on local ISM properties. We need to break this fundamental limit to properly study the KS relation at $z\gtrsim4$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.12516-b31b1b.svg)](https://arxiv.org/abs/2508.12516) | **The JWST Rocky Worlds DDT Program reveals GJ 3929b to likely be a bare rock**  |
|| Q. Xue, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-08-19*|
|*Comments*| *Submitted To AAS*|
|**Abstract**|            We report first results from the JWST Rocky Worlds Director's Discretionary Time program. Two secondary eclipses of the terrestrial exoplanet GJ 3929b were recently observed using MIRI photometric imaging at 15 um. We present a reduction of these data using the updated SPARTA pipeline. We also refine the planet mass, radius, and predicted time of secondary eclipse using a new sector of TESS data and new, high-precision radial velocities from the MAROON-X spectrograph. For the two JWST observations, we recover secondary eclipse depths of 177+47-45ppm and 143+34-35ppm at times consistent with a nearly circular orbit, as expected from the radial velocity data. A joint fit of the two visits yields a dayside brightness temperature Tp,dayside = 782+/-79K for GJ 3929b, which is consistent with the maximum brightness temperature Tmax = 737+/-14K for a bare, black rock (i.e., assuming zero Bond albedo and no heat redistribution). These results rule out CO2-rich atmospheres thicker than 100mbar at >3sigma, suggesting that GJ 3929b has lost any significant secondary atmosphere. The radial velocity data also indicate two additional non-transiting planets in the system: a previously-identified planet in a 15.0d orbit, and a newly-identified planet candidate in a 6.1d orbit.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.11761/./CO_blowup.png', 'tmp_2508.11761/./spectrum_new.png', '', '', '', '']
copying  tmp_2508.11761/./CO_blowup.png to _build/html/
copying  tmp_2508.11761/./spectrum_new.png to _build/html/
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2508.11761.md
    + _build/html/tmp_2508.11761/./CO_blowup.png
    + _build/html/tmp_2508.11761/./spectrum_new.png
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2508.12601/./Figures/fig3.png', 'tmp_2508.12601/./Figures/fig6.png', 'tmp_2508.12601/./Figures/fig1.png']
copying  tmp_2508.12601/./Figures/fig3.png to _build/html/
copying  tmp_2508.12601/./Figures/fig6.png to _build/html/
copying  tmp_2508.12601/./Figures/fig1.png to _build/html/
exported in  _build/html/2508.12601.md
    + _build/html/tmp_2508.12601/./Figures/fig3.png
    + _build/html/tmp_2508.12601/./Figures/fig6.png
    + _build/html/tmp_2508.12601/./Figures/fig1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ikc}[1]{\textcolor{green}{\textsf{#1 }}}$
$\newcommand{\jk}[1]{\textcolor{purple}{\rm #1^{(\rm new)}}}$</div>



<div id="title">

# MINDS. Strong oxygen depletion in the inner regions of a very low-mass star disk?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.11761-b31b1b.svg)](https://arxiv.org/abs/2508.11761)<mark>Appeared on: 2025-08-19</mark> -  _accepted for publication in AandA_

</div>
<div id="authors">

J. Kanwar, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** JWST is discovering a plethora of species in planet-forming disks around very low-mass stars such as $\ce{C2H2}$ , $\ce{C6H6}$ , $\ce{C4H2}$ , $\ce{CH3}$ etc. The column densities of these species retrieved from 0D slab models are very large, e.g. of the order of $10^{20}$ cm $^{-2}$ . This is indicating a carbon-dominated chemistry in a gas with a high C/O ratio. The disk around 2MASS-J1605321-1993159 (M4.5) is one such source showing a molecular pseudo-continuum of $\ce{C2H2}$ . Still two oxygen-bearing molecules, CO and $\ce{CO2}$ are also detected in this source. We aim to take the next step beyond 0D slab models to interpret the spectrum. We examine whether 2D thermo-chemical disk models can produce the large inferred column densities of $\ce{C2H2}$ in the inner regions of the disk and produce a pseudo-continuum in the mid-IR spectrum. We also want to constrain  whether depletion of oxygen or enrichment of carbon is causing the high C/O ratio triggering a carbon-dominated chemistry. We utilize the radiative thermo-chemical disk model P ${\tiny RO}$ D ${\tiny I}$ M ${\tiny O}$ to identify a disk structure which is capable of producing the observed molecular emission of species such as CO, $\ce{CO2}$ , $\ce{C2H2}$ , and $\ce{H2O}$ simultaneously. The spectrum is generated using the fast line tracer FLiTs. We derive the gas temperature $\langle T \rangle$ , column density $\langle$ log $_{\rm{10}} N\rangle$ and the emitting area $\langle r_{\rm{1}} - r_{\rm{2}} \rangle$ for these molecules from the 2D disk model and compare them to the parameters retrieved originally from 0D slab models. We use the different effect that changing the O or C abundance has on $\ce{CO}$ and $\ce{C2H2}$ respectively to discriminate between O depletion and C enhancement. We find that a disk structure characterised by the presence of a gap can best explain the observations. The inner disk is strongly depleted in dust, especially small grains ( $<5 \mu$ m), and elemental oxygen, leading to a large C/O ratio. This is required to produce a molecular pseudo-continuum of $\ce{C2H2}$ and at the same time a relatively weak $\ce{CO}$ emission. The P- and R-branch of $\ce{C2H2}$ probe deeper layers of the disk whereas the Q-branch probes mostly the surface layers. The combined emission of CO and $\ce{CO2}$ puts strong constraints on the gap's location (0.1-0.5 au) given a disk gas mass. We report here also the new detection of the CO $\nu$ = 2 $\rightarrow$ 1 transition in the JWST spectrum. 2D thermo-chemical disk models are able to produce the observed molecular pseudo-continuum of $\ce{C2H2}$ . We find that the combination of different species emission in the JWST spectra can be used to discriminate between different scenarios such as O-depletion, C-enhancement or both and offers the potential to extract spatial substructure at scales smaller than $\sim 1$ au.

</div>

<div id="div_fig1">

<img src="tmp_2508.11761/./CO_blowup.png" alt="Fig9" width="100%"/>

**Figure 9. -** The zoom-in JWST/MIRI MRS spectrum (black) showing CO emission. The emission of CO from the best thermo-chemical model convolved to 3500 and resampled to JWST-MIRI/MRS resolution is shown in brown. The H {\sc i} 10-6 line is marked in grey. (*CO*)

</div>
<div id="div_fig2">

<img src="tmp_2508.11761/./spectrum_new.png" alt="Fig23" width="100%"/>

**Figure 23. -** The modelled spectrum of the best disk model convolved and resampled to the JWST-MIRI/MRS resolution and compared to the continuum subtracted observed spectrum. \citet[Extended Data Figure 5]{Tabone2023} provide two different continua and therefore, we use the continuum from their middle panel to depict \ce{C2H2} and \ce{CH4} whereas the continuum from their bottom panel is used to depict the rest of the molecules. The solid black line depicts the region where the modeled fluxes approximately match the observations. These are the regions where little effect from blending is expected. (*spectrum*)

</div>
<div id="div_fig3">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.11761"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\Ou}{`Oumuamua}$
$\newcommand{\choh}{CH_3OH }$
$\newcommand{\hto}{H_2O }$
$\newcommand{\chisq}{\chi^2 }$
$\newcommand{\Av}{A_V }$
$\newcommand{\Ak}{A_K }$
$\newcommand{\cm}{cm^{-3} }$
$\newcommand{\mum}{\textmu m}$
$\newcommand{\arcm}{\mbox{\ensuremath{.\mkern-4mu^\prime}}}$
$\newcommand{\arcs}{\mbox{\ensuremath{.\!\!^{\prime\prime}}}}$
$\newcommand{\arcd}{\mbox{\ensuremath{.\!\!^\circ}}}$
$\newcommand{\degs}{\ensuremath{^{\circ}}}$
$\newcommand{\Lsun}{\ensuremath{L_\odot}}$
$\newcommand{\Msun}{\ensuremath{M_\odot}}$
$\newcommand{\eg}{e.g.,}$
$\newcommand{\ie}{i. e. }$
$\newcommand{\hawaii}{Hawai\!`i}$</div>



<div id="title">

# A Tale of Two Sightlines: Comparison of Hydrocarbon Dust Absorption Bands toward Cygnus OB2-12 and the Galactic Center

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.12601-b31b1b.svg)](https://arxiv.org/abs/2508.12601)<mark>Appeared on: 2025-08-19</mark> -  _28 pages, 7 figures, accepted by ApJ_

</div>
<div id="authors">

Y. J. Pendleton, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Infrared spectra of hydrocarbon dust absorption bands toward the bright hypergiant Cygnus OB2-12 are compared to published spectra of the Quintuplet Cluster, a sightline to the Galactic center. The Cyg OB2-12 data include a new ground-based 2.86 $-$ 3.70 $\mum$ spectrum and a previously published, but here further analyzed, spectrum of the 5.50--7.34 $\mum$ region. Higher spectral resolution data for the  Cyg OB2-12 sightline in the 3 $\mum$ region allows a detailed comparison of the 3.4 $\mum$ aliphatic bands to those observed toward the Quintuplet. Despite differences in interstellar environments along each sightline, strong similarities are observed  in the central wavelengths and relative strengths for bands at $\sim$ 3.3, 3.4, 5.85, 6.2, and 6.85 $\mum$ . Analysis of these bands, produced by aromatic, aliphatic, olefinic, hydrogenated, and oxygenated components, shows that carbonaceous dust is a significant component of the diffuse interstellar medium, second in abundance only to silicates, and is primarily aromatic in nature. The grains producing these bands likely consist of large aromatic carbon cores with thin aliphatic mantles composed of hydrogenated amorphous carbon (HAC). Laboratory analog spectra reproduce the observed aliphatic absorption bands well, supporting the presence of such mantles. We present evidence that the carriers of both the 3.4 $\mum$ aliphatic and the 3.3 $\mum$ aromatic bands reside exclusively in the diffuse ISM, and that the 3.3 $\mum$ bands observed in the diffuse ISM differ from those seen in dense clouds, implying chemically distinct carriers. $\keywords{astrochemistry---dust,extinction---ISM:molecules---ISM:lines and bands---infrared:ISM---techniques:spectroscopic}$

</div>

<div id="div_fig1">

<img src="tmp_2508.12601/./Figures/fig3.png" alt="Fig1" width="100%"/>

**Figure 1. -** Optical depth spectra of Cyg OB2-12 (black) and Quintuplet (blue) sightlines.  Both the near-IR and mid-IR optical depth spectra of the Quintuplet \cite[both from][]{chiar2013structure} are divided by 3.5 so that the strengths of the 3.4 $\mum$ match and thus facilitate comparison of relative band intensities. The continuum level of the mid-IR spectrum of the Quintuplet is slightly shifted from that in chiar2013structure as discussed in the text. (*fig:fig3*)

</div>
<div id="div_fig2">

<img src="tmp_2508.12601/./Figures/fig6.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
Comparison of spectral profiles of the 3.3 $\mum$ absorption bands towards Cyg OB2-12 (red) and the GC Quintuplet (blue; data from Figure \ref{fig:fig3}, this paper), and the 3.25 $\mum$ band in the deeply embedded YSO Mon R2 IRS3 \citep[][shown with its Gaussian fit, both in gray]{sellgren95}. The Gaussian used to fit the 3.3 $\mum$ bands in the spectra of Cyg OB2-12 and the Quintuplet is shown in black.  (*fig:fig6*)

</div>
<div id="div_fig3">

<img src="tmp_2508.12601/./Figures/fig1.png" alt="Fig3" width="100%"/>

**Figure 3. -** The 2.86$-$3.70 $\mum$ optical depth spectrum of Cyg OB2-12 obtained with SpeX on the NASA IRTF. The spectrum is displayed at a resolution of 0.004 $\mum$. Telluric line residuals were removed as described in \S \ref{sec: NIR obs}. Three $\ion${H}{1}  Pfund series (_ n_$-$5) absorption lines and a weak $\ion${He}{2} emission line are labeled. The wavelengths of the  $\ion${H}{1}  Humphreys series (_ n_$-$6) lines and the series limit at 3.283 $\mum$ are indicated by short vertical lines. Data points in the interval  3.31$-$3.32 $\mum$ are spurious  due to incomplete correction of the nearly totally opaque telluric absorption by closely spaced lines of the methane $\nu_{3}$ Q-branch, and are not shown. The noise level varies with wavelength but can be estimated from the point to point fluctuations in the signal in regions devoid of absorption features, such as 2.90$-$3.00 $\mum$ and 3.12$-$3.20 $\mum$. (*fig:fig1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.12601"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

129  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
